In [63]:
import pandas as pd
import glob
import os
import shutil
from joblib import Parallel, delayed
import nibabel as nib
import numpy as np
import random
import seaborn as sns
from datetime import datetime

In [64]:
import sys
sys.path.append('/media/raid/orco/data/ICH/code/neurorad_bids_utils')
from bids_utils.utils import utils, mask_utils

In [65]:
dataset_names = ['Dataset011_5FoldCV', 'Dataset012_5FoldCV', 'Dataset013_5FoldCV', 'Dataset014_5FoldCV', 'Dataset015_5FoldCV']

In [66]:
preds = glob.glob('/home/orco/data/VentrikelCNN/derivatives/nnUNet/nnUNet_raw/Dataset01*/test*/*.nii.gz')
len(preds)

154

In [67]:
df = pd.DataFrame(preds)
df.columns = ['prediction']
df['gt'] = df.prediction.apply(lambda x: x.replace('test_fold1', 'labelsTs'))
df['ds'] = df.prediction.apply(lambda x: x.split('/')[-3])

In [68]:
res = []
for i,j in zip(list(df.prediction),list(df['gt'])):
    res.append([i,j] + utils.mp_vol_nii_n(i, [1,2,3,4,5], n_jobs=50) + utils.mp_vol_nii_n(j, [1,2,3,4,5], n_jobs=50))

In [69]:
df = pd.DataFrame(res)

In [70]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,92458.659311,118653.143706,2664.931841,8006.660760,152.666027,86903.356158,128066.230353,2649.902543,7909.365830,174.814466
1,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,104875.232512,189070.150966,1843.857561,9935.948014,98.085945,104435.427791,190548.559280,1867.588031,9535.694078,117.070321
2,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,21495.851229,12167.403261,1137.480554,223.066423,31.640627,21399.347316,11572.559466,1133.525476,261.826192,8.701173
3,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,19334.005363,20935.021109,1343.935648,1535.361444,34.804690,19177.384258,21873.165711,1033.857500,1643.730593,30.849612
4,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,17198.263015,145104.708200,878.027410,1595.478636,3.955078,14671.758918,147842.413484,772.822324,1410.380966,0.000000


In [71]:
df.columns = ['prediction', 'gt', 'Vol_left', 'Vol_right', 'Vol_3rd', 'Vol_4th', 'Vol_Acq', 'gt_Vol_left', 'gt_Vol_right', 'gt_Vol_3rd', 'gt_Vol_4th', 'gt_Vol_Acq']


In [72]:
df['sid'] = df.prediction.apply(lambda x: os.path.basename(x).split('_')[1].replace('.nii.gz',''))

In [73]:
df['sid'] = 'sub-V' + df.sid

In [74]:
df.head()

,prediction,gt,Vol_left,Vol_right,Vol_3rd,Vol_4th,Vol_Acq,gt_Vol_left,gt_Vol_right,gt_Vol_3rd,gt_Vol_4th,gt_Vol_Acq,sid
0,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,92458.659311,118653.143706,2664.931841,8006.660760,152.666027,86903.356158,128066.230353,2649.902543,7909.365830,174.814466,sub-V083
1,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,104875.232512,189070.150966,1843.857561,9935.948014,98.085945,104435.427791,190548.559280,1867.588031,9535.694078,117.070321,sub-V108
2,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,21495.851229,12167.403261,1137.480554,223.066423,31.640627,21399.347316,11572.559466,1133.525476,261.826192,8.701173,sub-V115
3,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,19334.005363,20935.021109,1343.935648,1535.361444,34.804690,19177.384258,21873.165711,1033.857500,1643.730593,30.849612,sub-V074
4,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,17198.263015,145104.708200,878.027410,1595.478636,3.955078,14671.758918,147842.413484,772.822324,1410.380966,0.000000,sub-V266


In [75]:
frees_etivs = '/home/orco/data/VentrikelCNN/code/stats/freesurfer_eTIVs.csv'
etivs = pd.read_csv(frees_etivs)

In [76]:
etivs.head()

,path,eTIV
0,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,468381
1,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,1399601
2,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,474830
3,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,959807
4,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,793015


In [77]:
etivs['sid'] = etivs.path.apply(lambda x: x.split('/')[9])
etivs.head()                   

,path,eTIV,sid
0,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,468381,sub-V001
1,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,1399601,sub-V002
2,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,474830,sub-V003
3,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,959807,sub-V004
4,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,793015,sub-V005


In [78]:
df = df.merge(etivs, on='sid').drop(columns='path')
df = df[['sid', 'eTIV'] +[i for i in df.columns if i.startswith('Vol') or i.startswith('gt_Vol')]]
df.head()

,sid,eTIV,Vol_left,Vol_right,Vol_3rd,Vol_4th,Vol_Acq,gt_Vol_left,gt_Vol_right,gt_Vol_3rd,gt_Vol_4th,gt_Vol_Acq
0,sub-V083,1254638,92458.659311,118653.143706,2664.931841,8006.660760,152.666027,86903.356158,128066.230353,2649.902543,7909.365830,174.814466
1,sub-V108,408478,104875.232512,189070.150966,1843.857561,9935.948014,98.085945,104435.427791,190548.559280,1867.588031,9535.694078,117.070321
2,sub-V115,1218545,21495.851229,12167.403261,1137.480554,223.066423,31.640627,21399.347316,11572.559466,1133.525476,261.826192,8.701173
3,sub-V074,1300745,19334.005363,20935.021109,1343.935648,1535.361444,34.804690,19177.384258,21873.165711,1033.857500,1643.730593,30.849612
4,sub-V266,1646525,17198.263015,145104.708200,878.027410,1595.478636,3.955078,14671.758918,147842.413484,772.822324,1410.380966,0.000000


In [81]:
date = '{}{}{}'.format(datetime.now().year, datetime.now().month, datetime.now().day)
date

'2024319'

In [82]:
df.to_csv('./data/volumes_nnUNet_GT_5classes_{}_{}.csv'.format('5FoldCV', date), index=False)